# V0 - V3

In [ ]:
%pip install openreview-py numpy pandas nltk transformers tqdm torch
%pip install torch torchvision torchaudio
%pip install pylats taaled spacy convokit

# English models
# %python -m spacy download en_core_web_sm
# %python -m spacy download en_core_web_trf
# # Spanish models (used as fallback)
# %python -m spacy download es_core_news_sm
# %python -m spacy download es_dep_news_trf

In [1]:
import torch
print("Is GPU available? ", torch.cuda.is_available())

Is GPU available?  True


In [2]:
import json
import csv
from datetime import datetime
import pickle
import sys
import io
import re
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')
import csv
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm
from taaled import ld
from pylats import lats
import pandas as pd
import json
import pickle
import numpy as np


# ICLR 2024
# file_path = '/home/ali/Review_Quality_Benchmark/data/raw/OpenReview/ICLR2024/ICLR2024_submissions.pkl'

# NEURIPS 2023
file_path = '/home/ali/Review_Quality_Benchmark/data/raw/OpenReview/neurips2023/neurips2023_submissions.pkl'

with open(file_path, 'rb') as pkl_file:
    data = pickle.load(pkl_file)



[nltk_data] Downloading package punkt to /home/ali/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/ali/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
/home/ali/Review_Quality_Benchmark/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


plotnine has not been installed.
To enable advanced data visualization features, please install plotnine.
Attempting to load spacy model: en_core_web_sm
Successfully loaded spacy model: en_core_web_sm
Attempting to load spacy model: en_core_web_trf
Successfully loaded spacy model: en_core_web_trf
Attempting to load spacy model: es_core_news_sm
Successfully loaded spacy model: es_core_news_sm
Attempting to load spacy model: es_dep_news_trf
Successfully loaded spacy model: es_dep_news_trf


In [4]:
import pandas as pd
import json
import pickle
import numpy as np


# Replace 'your_file.pkl' with the path to your .pkl file
file_path = '/home/ali/Review_Quality_Benchmark/data/raw/OpenReview/neurips2023/neurips2023_submissions.pkl'
output_json_path = '/home/ali/Review_Quality_Benchmark/data/processed/openreview_neurips2023_v0.json'

# Load the .pkl file
with open(file_path, 'rb') as pkl_file:
    data = pickle.load(pkl_file)

# Extract the required fields for each submission
extracted_data = []
for submission in data:
    extracted_data.append({
        'number': submission.number if hasattr(submission, 'number') else np.nan,
        'id': submission.id if hasattr(submission, 'id') else np.nan,
        'content.paperhash': submission.content['paperhash']['value'] if 'paperhash' in submission.content and 'value' in submission.content['paperhash'] else np.nan,
        'content.authorids': submission.content['authorids']['value'] if 'authorids' in submission.content and 'value' in submission.content['authorids'] else np.nan,
        'cdate': submission.cdate if hasattr(submission, 'cdate') else np.nan,
        'content.title': submission.content['title']['value'] if 'title' in submission.content and 'value' in submission.content['title'] else np.nan,
        'content.abstract': submission.content['abstract']['value'] if 'abstract' in submission.content and 'value' in submission.content['abstract'] else np.nan,
        # 'content.TLDR': submission.content['TLDR']['value'] if 'TLDR' in submission.content and 'value' in submission.content['TLDR'] else np.nan,
    })

# Create a Pandas DataFrame
df = pd.DataFrame(extracted_data)

# Save the DataFrame to a JSON file
df.to_json(output_json_path, orient='records', indent=4)

print(f"DataFrame saved to JSON file at: {output_json_path}")

DataFrame saved to JSON file at: /home/ali/Review_Quality_Benchmark/data/processed/openreview_neurips2023_v0.json


In [5]:
# Load the JSON file as a pandas DataFrame
df_json = pd.read_json('/home/ali/Review_Quality_Benchmark/data/processed/openreview_neurips2023_v0.json')

# Display the header of the first 5 samples
df_json

,number,id,content.paperhash,content.authorids,cdate,content.title,content.abstract
0,10819,zyhxRc9bew,sun|what_is_flagged_in_uncertainty_quantificat...,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",1683789038840,What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...
1,15594,zyZkaqNnpa,puli|dont_blame_dataset_shift_shortcut_learnin...,"[~Aahlad_Manas_Puli1, ~Lily_H_Zhang1, ~Yoav_Wa...",1683835167534,Don’t blame Dataset Shift! Shortcut Learning d...,Common explanations for shortcut learning assu...
2,327,zuXyQsXVLF,xu|enhancing_adversarial_contrastive_learning_...,"[~Xilie_Xu1, ~Jingfeng_Zhang1, ~Feng_Liu2, ~Ma...",1682066194522,Enhancing Adversarial Contrastive Learning via...,Adversarial contrastive learning (ACL) is a te...
3,801,ztqf6bzuqQ,shi|hybrid_distillation_connecting_masked_auto...,"[~Bowen_Shi2, ~XIAOPENG_ZHANG7, ~Yaoming_Wang1...",1682507396411,Hybrid Distillation: Connecting Masked Autoenc...,Representation learning has been evolving from...
4,7411,ztDxO15N7f,scholkemper|an_optimizationbased_approach_to_n...,"[~Michael_Scholkemper1, ~Michael_T_Schaub1]",1683729208530,An Optimization-based Approach To Node Role Di...,"Similar to community detection, partitioning t..."
...,...,...,...,...,...,...,...
3390,12557,05P1U0jk8r,sakos|exploiting_hidden_structures_in_nonconve...,"[~Iosif_Sakos1, ~Emmanouil-Vasileios_Vlatakis-...",1683812282551,Exploiting hidden structures in non-convex gam...,A wide array of modern machine learning applic...
3391,12654,02Uc0G2Cym,mianjy|robustness_guarantees_for_adversarially...,"[~Poorya_Mianjy1, ~Raman_Arora1]",1683813240757,Robustness Guarantees for Adversarially Traine...,We study robust adversarial training of two-la...
3392,1581,01GQK1gwe3,holl|can_neural_networks_improve_classical_opt...,"[~Philipp_Holl1, ~Nils_Thuerey1]",1683105912880,Can Neural Networks Improve Classical Optimiza...,Finding the values of model parameters from da...
3393,6838,00EKYYu3fD,hu|complexity_matters_rethinking_the_latent_sp...,"[~Tianyang_Hu1, ~Fei_Chen8, ~Haonan_Wang1, ~Ji...",1683719044332,Complexity Matters: Rethinking the Latent Spac...,"In generative modeling, numerous successful ap..."


In [6]:
# Print the dimensions of the dataframe
print(f"Dimensions of the dataframe: {df.shape}")

# Count the number of NaN values in each column
print("Number of NaN values in each column:")
print(df.isna().sum())

Dimensions of the dataframe: (3395, 7)
Number of NaN values in each column:
number               0
id                   0
content.paperhash    0
content.authorids    0
cdate                0
content.title        0
content.abstract     0
dtype: int64


In [8]:
# number of review, comments, and meta-reviews for each submission
# 3395 is the number of submissions
num_of_reviews = [len(data[i].details['directReplies']) for i in range(3395)]
print('min:', min(num_of_reviews))
print('max:', max(num_of_reviews))
print('mean:', np.mean(num_of_reviews))
print('median:', np.median(num_of_reviews))
print('std:', np.std(num_of_reviews))
print('percentiles:', np.percentile(num_of_reviews, [25, 50, 75, 90, 95, 99]))

min: 4
max: 14
mean: 6.455670103092784
median: 6.0
std: 0.905242195033333
percentiles: [6. 6. 7. 8. 8. 9.]


In [10]:
comment_types = set()
for i in range(3395):
    for j in range(len(data[i].details['directReplies'])):
        comment_types.add(data[i].details['directReplies'][j]['invitations'][0].split('/')[-1])

comment_types

{'Author_Rebuttal', 'Decision', 'Official_Comment', 'Official_Review'}

In [11]:
# Iterate through each submission in the data
new_rows = []
for submission in data:
    submission_id = submission.id
    submission_number = submission.number
    submission_title = submission.content['title']['value'] if 'title' in submission.content and 'value' in submission.content['title'] else None
    submission_abstract = submission.content['abstract']['value'] if 'abstract' in submission.content and 'value' in submission.content['abstract'] else None
    submission_authors = submission.content['authorids']['value'] if 'authorids' in submission.content and 'value' in submission.content['authorids'] else np.nan
    submission_creation_date = submission.cdate if hasattr(submission, 'cdate') else np.nan

    # Check if 'directReplies' exists in details
    if 'directReplies' in submission.details:
        for reply in submission.details['directReplies']:
            # Check if the invitation is 'Official_Review'
            if reply['invitations'][0].split('/')[-1] == 'Official_Review':
                # Extract features from the review
                reviewer = reply['signatures'][0].split('/')[-1]  # if 'signatures' in reply and len(reply['signatures']) > 0 else None
                
                #'tcdate', 'cdate', 'tmdate', 'mdate'
                review_tcdate = reply['tcdate'] if 'tcdate' in reply else None
                review_cdate = reply['cdate'] if 'cdate' in reply else None
                review_tmdate = reply['tmdate'] if 'tmdate' in reply else None
                review_mdate = reply['mdate'] if 'mdate' in reply else None
                
                review_rating = int(reply['content']['rating']['value'].split(':')[0]) if 'rating' in reply['content'] and 'value' in reply['content']['rating'] else None
                review_confidence = int(reply['content']['confidence']['value'].split(':')[0]) if 'confidence' in reply['content'] and 'value' in reply['content']['confidence'] else None
                review_soundness = int(reply['content']['soundness']['value'].split(' ')[0]) if 'soundness' in reply['content'] and 'value' in reply['content']['soundness'] else None
                review_presentation = int(reply['content']['presentation']['value'].split(' ')[0]) if 'presentation' in reply['content'] and 'value' in reply['content']['presentation'] else None
                review_contribution = int(reply['content']['contribution']['value'].split(' ')[0]) if 'contribution' in reply['content'] and 'value' in reply['content']['contribution'] else None
                
                review_summary = reply['content']['summary']['value'] if 'summary' in reply['content'] and 'value' in reply['content']['summary'] else None
                review_strengths = reply['content']['strengths']['value'] if 'strengths' in reply['content'] and 'value' in reply['content']['strengths'] else None
                review_weaknesses = reply['content']['weaknesses']['value'] if 'weaknesses' in reply['content'] and 'value' in reply['content']['weaknesses'] else None
                review_questions = reply['content']['questions']['value'] if 'questions' in reply['content'] and 'value' in reply['content']['questions'] else None
                review_limitations = reply['content']['limitations']['value'] if 'limitations' in reply['content'] and 'value' in reply['content']['limitations'] else None
                
                # Create a new row with the extracted features
                new_row = {
                    'submission_id': submission_id,
                    'submission_number': submission_number,
                    'submission_creation_date': submission_creation_date,
                    'submission_authors': submission_authors,
                    
                    'submission_title': submission_title,
                    'submission_abstract': submission_abstract,
                    
                    'reviewer': reviewer,
                    'review_tcdate': review_tcdate,
                    'review_cdate': review_cdate,
                    'review_tmdate': review_tmdate,
                    'review_mdate': review_mdate,
                    
                    'review_summary': review_summary,
                    'review_strengths': review_strengths,
                    'review_weaknesses': review_weaknesses,
                    'review_questions': review_questions,
                    'review_limitations': review_limitations,
                    
                    'review_rating': review_rating,
                    'review_confidence': review_confidence,
                    'review_soundness': review_soundness,
                    'review_presentation': review_presentation,
                    'review_contribution': review_contribution
                }
                new_rows.append(new_row)

# Create a new DataFrame from the new rows
df_reviews = pd.DataFrame(new_rows)

# Save the updated DataFrame to a JSON file
output_json_path = '/home/ali/Review_Quality_Benchmark/data/processed/openreview_neurips2023_v1.json'
df_reviews.to_json(output_json_path, orient='records', indent=4)

print(f"Updated DataFrame with reviews saved to JSON file at: {output_json_path}")

Updated DataFrame with reviews saved to JSON file at: /home/ali/Review_Quality_Benchmark/data/processed/openreview_neurips2023_v1.json


In [12]:
# Load the JSON file as a pandas DataFrame
df_reviews = pd.read_json('/home/ali/Review_Quality_Benchmark/data/processed/openreview_neurips2023_v1.json')

# Display the header of the first 5 samples
df_reviews

,submission_id,submission_number,submission_creation_date,submission_authors,submission_title,submission_abstract,reviewer,review_tcdate,review_cdate,review_tmdate,...,review_summary,review_strengths,review_weaknesses,review_questions,review_limitations,review_rating,review_confidence,review_soundness,review_presentation,review_contribution
0,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_AvJq,1688368213177,1688368213177,1702411303415,...,This paper proposes a model-agnostic framework...,1. This paper focuses on samples that remain s...,1. Why are the uncertain samples of the model ...,"1. Checking some details in the paper, such as...",Is DAUC a model-agnostic approach that can be ...,6,3,3,3,3
1,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_7E4k,1688505633161,1688505633161,1702411303319,...,"In this paper, the authors present a unique ap...","In my opinion, this categorization process of ...",See questions.,"* In my opinion, at a high level, this method ...","Yes, the limitations have been listed by the a...",6,4,3,3,4
2,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_hHZH,1688552936677,1688552936677,1702411303221,...,I have read the other reviews and all rebuttal...,"The paper is very well written overall, I enjo...",I find it quite difficult to judge the technic...,1. Can the proposed method be applied to datas...,Yes.,6,4,3,3,2
3,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_sVgx,1688657604892,1688657604892,1702411303144,...,Paper introduces a framework to detect and cat...,"To my knowledge, paper presents a novel approa...",There are some weaknesses that could further i...,- Table 1: Could you ellaborate why other rela...,Some of the assumptions and limitations of pro...,6,4,2,3,3
4,zyZkaqNnpa,15594,1683835167534,"[~Aahlad_Manas_Puli1, ~Lily_H_Zhang1, ~Yoav_Wa...",Don’t blame Dataset Shift! Shortcut Learning d...,Common explanations for shortcut learning assu...,Reviewer_9QKU,1688617232745,1688617232745,1702411520565,...,This paper theoretecally and empirically showe...,This paper analyzes shortcut learning theoreti...,The theory itself has limited applicability du...,It seems that this paper forcus on a theoretic...,The theory itself has limited applicability du...,6,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15170,00EKYYu3fD,6838,1683719044332,"[~Tianyang_Hu1, ~Fei_Chen8, ~Haonan_Wang1, ~Ji...",Complexity Matters: Rethinking the Latent Spac...,"In generative modeling, numerous successful ap...",Reviewer_ssMC,1688676041356,1688676041356,1702411081106,...,This paper proposes an asymmetric training sch...,The strengths of this paper are :\n- It is a s...,The main weakness is that most of the paper is...,l.146 the link between the ideal $P_z$ and a l...,The conclusion is a list of limitations that i...,6,4,3,3,3
15171,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_UdXn,1688449656890,1688449656890,1702411268900,...,"In this paper, authors present regression adju...",The main strength of this paper is a finite sa...,"The paper has a few minor issues, but I believ...",- Can you address my comment on conditional ve...,yes,7,5,3,3,3
15172,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and 

In [13]:
# Concatenate the specified columns into a single column named 'total_review'
df_reviews['total_review'] = df_reviews[['review_summary', 'review_strengths', 'review_weaknesses', 'review_questions', 'review_limitations']].apply(
    lambda row: ' '.join(row.dropna()), axis=1
)

# Drop the original columns to reduce redundancy
df_reviews = df_reviews.drop(columns=['review_summary', 'review_strengths', 'review_weaknesses', 'review_questions', 'review_limitations'])
# Create a new column 'length_words' to count the number of words in the 'total_review' column
df_reviews['length_words'] = df_reviews['total_review'].apply(lambda x: len(x.split()))


# Save the new DataFrame to a different JSON file
new_output_json_path = '/home/ali/Review_Quality_Benchmark/data/processed/openreview_ICLR2024_total_review.json'
df_reviews.to_json(new_output_json_path, orient='records', indent=4)

print(f"New DataFrame with 'total_review' column saved to JSON file at: {new_output_json_path}")

New DataFrame with 'total_review' column saved to JSON file at: /home/ali/Review_Quality_Benchmark/data/processed/openreview_ICLR2024_total_review.json


In [15]:
df_reviews

,submission_id,submission_number,submission_creation_date,submission_authors,submission_title,submission_abstract,reviewer,review_tcdate,review_cdate,review_tmdate,review_mdate,review_rating,review_confidence,review_soundness,review_presentation,review_contribution,total_review,length_words
0,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_AvJq,1688368213177,1688368213177,1702411303415,1702411303415,6,3,3,3,3,This paper proposes a model-agnostic framework...,225
1,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_7E4k,1688505633161,1688505633161,1702411303319,1702411303319,6,4,3,3,4,"In this paper, the authors present a unique ap...",513
2,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_hHZH,1688552936677,1688552936677,1702411303221,1702411303221,6,4,3,3,2,I have read the other reviews and all rebuttal...,650
3,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_sVgx,1688657604892,1688657604892,1702411303144,1702411303144,6,4,2,3,3,Paper introduces a framework to detect and cat...,453
4,zyZkaqNnpa,15594,1683835167534,"[~Aahlad_Manas_Puli1, ~Lily_H_Zhang1, ~Yoav_Wa...",Don’t blame Dataset Shift! Shortcut Learning d...,Common explanations for shortcut learning assu...,Reviewer_9QKU,1688617232745,1688617232745,1702411520565,1702411520565,6,3,3,3,3,This paper theoretecally and empirically showe...,307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15170,00EKYYu3fD,6838,1683719044332,"[~Tianyang_Hu1, ~Fei_Chen8, ~Haonan_Wang1, ~Ji...",Complexity Matters: Rethinking the Latent Spac...,"In generative modeling, numerous successful ap...",Reviewer_ssMC,1688676041356,1688676041356,1702411081106,1702411081106,6,4,3,3,3,This paper proposes an asymmetric training sch...,574
15171,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_UdXn,1688449656890,1688449656890,1702411268900,1702411268900,7,5,3,3,3,"In this paper, authors present regression adju...",1063
15172,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_ZgSr,1688485585833,1688485585833,1702411268818,1702411268818,5,3,2,2,2,This paper focuses on estimation of individual...,519
15173,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_HGb9,1688665406904,1688665406904,1702411268706,1702411268706,4,3,2,1,3,This paper explores the design and analysis of...,1546


In [16]:
import re


def count_citations(text):
    citation_patterns = [
        r'\[\d+(?:,\s*\d+)*\]',                         # [1], [1, 2, 3]
        r'\([A-Za-z]+ et al\.,\s*\d{4}\)',               # (Smith et al., 2020)
        r'\(\d{4}[a-z]?\)',                              # (2020), (2020a)
        r'\[[A-Za-z]+\d{4}[a-z]?\]',                     # [Smith2020], [Johnson2021a]
        r'\b(?:doi:|arxiv:|https?://[^\s]+)',             # DOI, arXiv, URLs
    ]
    pattern = '|'.join(citation_patterns)
    matches = re.findall(pattern, text)
    return len(matches)


# Apply the count_citations function to the 'total_review' column and create a new column 'citation_count'
df_reviews['citation_count'] = df_reviews['total_review'].apply(count_citations)
df_reviews

,submission_id,submission_number,submission_creation_date,submission_authors,submission_title,submission_abstract,reviewer,review_tcdate,review_cdate,review_tmdate,review_mdate,review_rating,review_confidence,review_soundness,review_presentation,review_contribution,total_review,length_words,citation_count
0,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_AvJq,1688368213177,1688368213177,1702411303415,1702411303415,6,3,3,3,3,This paper proposes a model-agnostic framework...,225,0
1,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_7E4k,1688505633161,1688505633161,1702411303319,1702411303319,6,4,3,3,4,"In this paper, the authors present a unique ap...",513,0
2,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_hHZH,1688552936677,1688552936677,1702411303221,1702411303221,6,4,3,3,2,I have read the other reviews and all rebuttal...,650,0
3,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_sVgx,1688657604892,1688657604892,1702411303144,1702411303144,6,4,2,3,3,Paper introduces a framework to detect and cat...,453,0
4,zyZkaqNnpa,15594,1683835167534,"[~Aahlad_Manas_Puli1, ~Lily_H_Zhang1, ~Yoav_Wa...",Don’t blame Dataset Shift! Shortcut Learning d...,Common explanations for shortcut learning assu...,Reviewer_9QKU,1688617232745,1688617232745,1702411520565,1702411520565,6,3,3,3,3,This paper theoretecally and empirically showe...,307,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15170,00EKYYu3fD,6838,1683719044332,"[~Tianyang_Hu1, ~Fei_Chen8, ~Haonan_Wang1, ~Ji...",Complexity Matters: Rethinking the Latent Spac...,"In generative modeling, numerous successful ap...",Reviewer_ssMC,1688676041356,1688676041356,1702411081106,1702411081106,6,4,3,3,3,This paper proposes an asymmetric training sch...,574,0
15171,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_UdXn,1688449656890,1688449656890,1702411268900,1702411268900,7,5,3,3,3,"In this paper, authors present regression adju...",1063,1
15172,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_ZgSr,1688485585833,1688485585833,1702411268818,1702411268818,5,3,2,2,2,This paper focuses on estimation of individual...,519,3
15173,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_HGb9,1688665406904,1688665406904,1702411268706,1702411268706,4,3,2,1,3,This paper explores the design and analysis of...,1546,3


In [18]:
import csv
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("shahrukhx01/bert-mini-finetune-question-detection")
model = AutoModelForSequenceClassification.from_pretrained("shahrukhx01/bert-mini-finetune-question-detection")
model.to(device)
model.eval()


def count_questions(review_text):
    
    if review_text:
        question_count = 0

        sentences = sent_tokenize(review_text)
        for sent in sentences:
            inputs = tokenizer(
                sent,
                return_tensors="pt",
                truncation=True,
                max_length=64,
                padding=True
            ).to(device)
            with torch.no_grad():
                outputs = model(**inputs)
                predicted = torch.argmax(outputs.logits, dim=1).item()

                # Label 0 = question
                if predicted == 0:
                    question_count += 1

    return question_count


df_reviews['question_count'] = [
    count_questions(row['total_review']) for row in tqdm(df_reviews.to_dict('records'), desc="Processing reviews")
]
df_reviews

2025-05-08 14:15:26.522604: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746728126.544830 3911350 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746728126.551650 3911350 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746728126.571549 3911350 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746728126.571568 3911350 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746728126.571570 3911350 computation_placer.cc:177] computation placer alr

,submission_id,submission_number,submission_creation_date,submission_authors,submission_title,submission_abstract,reviewer,review_tcdate,review_cdate,review_tmdate,review_mdate,review_rating,review_confidence,review_soundness,review_presentation,review_contribution,total_review,length_words,citation_count,question_count
0,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_AvJq,1688368213177,1688368213177,1702411303415,1702411303415,6,3,3,3,3,This paper proposes a model-agnostic framework...,225,0,8
1,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_7E4k,1688505633161,1688505633161,1702411303319,1702411303319,6,4,3,3,4,"In this paper, the authors present a unique ap...",513,0,1
2,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_hHZH,1688552936677,1688552936677,1702411303221,1702411303221,6,4,3,3,2,I have read the other reviews and all rebuttal...,650,0,6
3,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_sVgx,1688657604892,1688657604892,1702411303144,1702411303144,6,4,2,3,3,Paper introduces a framework to detect and cat...,453,0,1
4,zyZkaqNnpa,15594,1683835167534,"[~Aahlad_Manas_Puli1, ~Lily_H_Zhang1, ~Yoav_Wa...",Don’t blame Dataset Shift! Shortcut Learning d...,Common explanations for shortcut learning assu...,Reviewer_9QKU,1688617232745,1688617232745,1702411520565,1702411520565,6,3,3,3,3,This paper theoretecally and empirically showe...,307,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15170,00EKYYu3fD,6838,1683719044332,"[~Tianyang_Hu1, ~Fei_Chen8, ~Haonan_Wang1, ~Ji...",Complexity Matters: Rethinking the Latent Spac...,"In generative modeling, numerous successful ap...",Reviewer_ssMC,1688676041356,1688676041356,1702411081106,1702411081106,6,4,3,3,3,This paper proposes an asymmetric training sch...,574,0,3
15171,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_UdXn,1688449656890,1688449656890,1702411268900,1702411268900,7,5,3,3,3,"In this paper, authors present regression adju...",1063,1,1
15172,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_ZgSr,1688485585833,1688485585833,1702411268818,1702411268818,5,3,2,2,2,This paper focuses on estimation of individual...,519,3,2
15173,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_HGb9,1688665406904,1688665406904,1702411268706,1702411268706,4,3,2,1,3,This paper explores the design and analysis of...,1546,3,9


In [19]:
df_reviews

,submission_id,submission_number,submission_creation_date,submission_authors,submission_title,submission_abstract,reviewer,review_tcdate,review_cdate,review_tmdate,review_mdate,review_rating,review_confidence,review_soundness,review_presentation,review_contribution,total_review,length_words,citation_count,question_count
0,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_AvJq,1688368213177,1688368213177,1702411303415,1702411303415,6,3,3,3,3,This paper proposes a model-agnostic framework...,225,0,8
1,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_7E4k,1688505633161,1688505633161,1702411303319,1702411303319,6,4,3,3,4,"In this paper, the authors present a unique ap...",513,0,1
2,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_hHZH,1688552936677,1688552936677,1702411303221,1702411303221,6,4,3,3,2,I have read the other reviews and all rebuttal...,650,0,6
3,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_sVgx,1688657604892,1688657604892,1702411303144,1702411303144,6,4,2,3,3,Paper introduces a framework to detect and cat...,453,0,1
4,zyZkaqNnpa,15594,1683835167534,"[~Aahlad_Manas_Puli1, ~Lily_H_Zhang1, ~Yoav_Wa...",Don’t blame Dataset Shift! Shortcut Learning d...,Common explanations for shortcut learning assu...,Reviewer_9QKU,1688617232745,1688617232745,1702411520565,1702411520565,6,3,3,3,3,This paper theoretecally and empirically showe...,307,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15170,00EKYYu3fD,6838,1683719044332,"[~Tianyang_Hu1, ~Fei_Chen8, ~Haonan_Wang1, ~Ji...",Complexity Matters: Rethinking the Latent Spac...,"In generative modeling, numerous successful ap...",Reviewer_ssMC,1688676041356,1688676041356,1702411081106,1702411081106,6,4,3,3,3,This paper proposes an asymmetric training sch...,574,0,3
15171,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_UdXn,1688449656890,1688449656890,1702411268900,1702411268900,7,5,3,3,3,"In this paper, authors present regression adju...",1063,1,1
15172,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_ZgSr,1688485585833,1688485585833,1702411268818,1702411268818,5,3,2,2,2,This paper focuses on estimation of individual...,519,3,2
15173,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_HGb9,1688665406904,1688665406904,1702411268706,1702411268706,4,3,2,1,3,This paper explores the design and analysis of...,1546,3,9


In [20]:
# Count rows where cdate and tcdate are equal or not equal
cdate_tcdate_equal = (df_reviews['review_cdate'] == df_reviews['review_tcdate']).sum()
cdate_tcdate_not_equal = (df_reviews['review_cdate'] != df_reviews['review_tcdate']).sum()

# Count rows where tmdate and mdate are equal or not equal
tmdate_mdate_equal = (df_reviews['review_tmdate'] == df_reviews['review_mdate']).sum()
tmdate_mdate_not_equal = (df_reviews['review_tmdate'] != df_reviews['review_mdate']).sum()

# Print the results
print(f"cdate and tcdate equal: {cdate_tcdate_equal}")
print(f"cdate and tcdate not equal: {cdate_tcdate_not_equal}")
print(f"tmdate and mdate equal: {tmdate_mdate_equal}")
print(f"tmdate and mdate not equal: {tmdate_mdate_not_equal}")

cdate and tcdate equal: 15175
cdate and tcdate not equal: 0
tmdate and mdate equal: 15175
tmdate and mdate not equal: 0


In [21]:
# Drop the 'tcdate' and 'tmdate' columns
df_reviews = df_reviews.drop(columns=['review_tcdate', 'review_tmdate'])

# Rename 'cdate' to 'creation_date' and 'mdate' to 'last_modification_date'
df_reviews = df_reviews.rename(columns={'review_cdate': 'creation_date', 'review_mdate': 'last_modification_date'})

# Display the updated DataFrame
df_reviews

,submission_id,submission_number,submission_creation_date,submission_authors,submission_title,submission_abstract,reviewer,creation_date,last_modification_date,review_rating,review_confidence,review_soundness,review_presentation,review_contribution,total_review,length_words,citation_count,question_count
0,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_AvJq,1688368213177,1702411303415,6,3,3,3,3,This paper proposes a model-agnostic framework...,225,0,8
1,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_7E4k,1688505633161,1702411303319,6,4,3,3,4,"In this paper, the authors present a unique ap...",513,0,1
2,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_hHZH,1688552936677,1702411303221,6,4,3,3,2,I have read the other reviews and all rebuttal...,650,0,6
3,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_sVgx,1688657604892,1702411303144,6,4,2,3,3,Paper introduces a framework to detect and cat...,453,0,1
4,zyZkaqNnpa,15594,1683835167534,"[~Aahlad_Manas_Puli1, ~Lily_H_Zhang1, ~Yoav_Wa...",Don’t blame Dataset Shift! Shortcut Learning d...,Common explanations for shortcut learning assu...,Reviewer_9QKU,1688617232745,1702411520565,6,3,3,3,3,This paper theoretecally and empirically showe...,307,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15170,00EKYYu3fD,6838,1683719044332,"[~Tianyang_Hu1, ~Fei_Chen8, ~Haonan_Wang1, ~Ji...",Complexity Matters: Rethinking the Latent Spac...,"In generative modeling, numerous successful ap...",Reviewer_ssMC,1688676041356,1702411081106,6,4,3,3,3,This paper proposes an asymmetric training sch...,574,0,3
15171,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_UdXn,1688449656890,1702411268900,7,5,3,3,3,"In this paper, authors present regression adju...",1063,1,1
15172,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_ZgSr,1688485585833,1702411268818,5,3,2,2,2,This paper focuses on estimation of individual...,519,3,2
15173,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_HGb9,1688665406904,1702411268706,4,3,2,1,3,This paper explores the design and analysis of...,1546,3,9


In [23]:
# Save the new DataFrame to a different JSON file
out = '/home/ali/Review_Quality_Benchmark/data/processed/openreview_neurips2023_v2.json'
df_reviews.to_json(out, orient='records', indent=4)

# V4 - V5

In [ ]:
%pip install spacy

# English models
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_trf
# # Spanish models (used as fallback)
!python -m spacy download es_core_news_sm
!python -m spacy download es_dep_news_trf

In [3]:
# Read the JSON file as a pandas DataFrame
df_reviews = pd.read_json('/home/ali/Review_Quality_Benchmark/data/processed/openreview_neurips2023_v2.json')

# Display the first few rows of the DataFrame
df_reviews

,submission_id,submission_number,submission_creation_date,submission_authors,submission_title,submission_abstract,reviewer,creation_date,last_modification_date,review_rating,review_confidence,review_soundness,review_presentation,review_contribution,total_review,length_words,citation_count,question_count
0,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_AvJq,1688368213177,1702411303415,6,3,3,3,3,This paper proposes a model-agnostic framework...,225,0,8
1,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_7E4k,1688505633161,1702411303319,6,4,3,3,4,"In this paper, the authors present a unique ap...",513,0,1
2,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_hHZH,1688552936677,1702411303221,6,4,3,3,2,I have read the other reviews and all rebuttal...,650,0,6
3,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_sVgx,1688657604892,1702411303144,6,4,2,3,3,Paper introduces a framework to detect and cat...,453,0,1
4,zyZkaqNnpa,15594,1683835167534,"[~Aahlad_Manas_Puli1, ~Lily_H_Zhang1, ~Yoav_Wa...",Don’t blame Dataset Shift! Shortcut Learning d...,Common explanations for shortcut learning assu...,Reviewer_9QKU,1688617232745,1702411520565,6,3,3,3,3,This paper theoretecally and empirically showe...,307,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15170,00EKYYu3fD,6838,1683719044332,"[~Tianyang_Hu1, ~Fei_Chen8, ~Haonan_Wang1, ~Ji...",Complexity Matters: Rethinking the Latent Spac...,"In generative modeling, numerous successful ap...",Reviewer_ssMC,1688676041356,1702411081106,6,4,3,3,3,This paper proposes an asymmetric training sch...,574,0,3
15171,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_UdXn,1688449656890,1702411268900,7,5,3,3,3,"In this paper, authors present regression adju...",1063,1,1
15172,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_ZgSr,1688485585833,1702411268818,5,3,2,2,2,This paper focuses on estimation of individual...,519,3,2
15173,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_HGb9,1688665406904,1702411268706,4,3,2,1,3,This paper explores the design and analysis of...,1546,3,9


In [4]:
from taaled import ld
from pylats import lats


def compute_mattr(review_text):
    mattr_value = ""
    if review_text is not None:
        review_text = review_text.strip()  # Remove leading/trailing whitespace
        review_text = review_text.replace('\n', '')  # Replace newlines with spaces
        try:
            cleaned = lats.Normalize(review_text, lats.ld_params_en)
            tokens = cleaned.toks
            mattr_value = f"{ld.lexdiv(tokens).mattr:.4f}"
        except Exception as e:
            mattr_value = ""
    return mattr_value


df_reviews['mattr'] = [
    compute_mattr(row['total_review']) for row in tqdm(df_reviews.to_dict('records'), desc="Processing reviews")
]
df_reviews

Processing reviews: 100%|██████████| 15175/15175 [40:43<00:00,  6.21it/s]  


,submission_id,submission_number,submission_creation_date,submission_authors,submission_title,submission_abstract,reviewer,creation_date,last_modification_date,review_rating,review_confidence,review_soundness,review_presentation,review_contribution,total_review,length_words,citation_count,question_count,mattr
0,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_AvJq,1688368213177,1702411303415,6,3,3,3,3,This paper proposes a model-agnostic framework...,225,0,8,0.7494
1,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_7E4k,1688505633161,1702411303319,6,4,3,3,4,"In this paper, the authors present a unique ap...",513,0,1,0.7788
2,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_hHZH,1688552936677,1702411303221,6,4,3,3,2,I have read the other reviews and all rebuttal...,650,0,6,0.7975
3,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_sVgx,1688657604892,1702411303144,6,4,2,3,3,Paper introduces a framework to detect and cat...,453,0,1,0.8136
4,zyZkaqNnpa,15594,1683835167534,"[~Aahlad_Manas_Puli1, ~Lily_H_Zhang1, ~Yoav_Wa...",Don’t blame Dataset Shift! Shortcut Learning d...,Common explanations for shortcut learning assu...,Reviewer_9QKU,1688617232745,1702411520565,6,3,3,3,3,This paper theoretecally and empirically showe...,307,0,0,0.8162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15170,00EKYYu3fD,6838,1683719044332,"[~Tianyang_Hu1, ~Fei_Chen8, ~Haonan_Wang1, ~Ji...",Complexity Matters: Rethinking the Latent Spac...,"In generative modeling, numerous successful ap...",Reviewer_ssMC,1688676041356,1702411081106,6,4,3,3,3,This paper proposes an asymmetric training sch...,574,0,3,0.7740
15171,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_UdXn,1688449656890,1702411268900,7,5,3,3,3,"In this paper, authors present regression adju...",1063,1,1,0.8006
15172,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_ZgSr,1688485585833,1702411268818,5,3,2,2,2,This paper focuses on estimation of individual...,519,3,2,0.7540
15173,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_HGb9,1688665406904,1702411268706,4,3,2,1,3,This paper explores the design and analysis of...,1546,3,9,0.7824


In [5]:
# Save the new DataFrame to a different JSON file
out = '/home/ali/Review_Quality_Benchmark/data/processed/openreview_neurips2023_v3.json'
df_reviews.to_json(out, orient='records', indent=4)

In [ ]:
%pip install textblob
!python -m textblob.download_corpora

In [7]:
import csv
from textblob import TextBlob
from tqdm import tqdm


def compute_sentiment_polarity(review_text):
    review_text = review_text.strip()
    try:
        blob = TextBlob(review_text)
        sentiment = blob.sentiment.polarity
    except Exception:
        sentiment = ""

    return sentiment


df_reviews['sentiment_polarity'] = [
    compute_sentiment_polarity(row['total_review']) for row in tqdm(df_reviews.to_dict('records'), desc="Processing reviews")
]
df_reviews

Processing reviews: 100%|██████████| 15175/15175 [00:29<00:00, 510.35it/s]


,submission_id,submission_number,submission_creation_date,submission_authors,submission_title,submission_abstract,reviewer,creation_date,last_modification_date,review_rating,review_confidence,review_soundness,review_presentation,review_contribution,total_review,length_words,citation_count,question_count,mattr,sentiment_polarity
0,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_AvJq,1688368213177,1702411303415,6,3,3,3,3,This paper proposes a model-agnostic framework...,225,0,8,0.7494,0.041667
1,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_7E4k,1688505633161,1702411303319,6,4,3,3,4,"In this paper, the authors present a unique ap...",513,0,1,0.7788,0.107136
2,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_hHZH,1688552936677,1702411303221,6,4,3,3,2,I have read the other reviews and all rebuttal...,650,0,6,0.7975,0.093388
3,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_sVgx,1688657604892,1702411303144,6,4,2,3,3,Paper introduces a framework to detect and cat...,453,0,1,0.8136,0.113038
4,zyZkaqNnpa,15594,1683835167534,"[~Aahlad_Manas_Puli1, ~Lily_H_Zhang1, ~Yoav_Wa...",Don’t blame Dataset Shift! Shortcut Learning d...,Common explanations for shortcut learning assu...,Reviewer_9QKU,1688617232745,1702411520565,6,3,3,3,3,This paper theoretecally and empirically showe...,307,0,0,0.8162,0.024399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15170,00EKYYu3fD,6838,1683719044332,"[~Tianyang_Hu1, ~Fei_Chen8, ~Haonan_Wang1, ~Ji...",Complexity Matters: Rethinking the Latent Spac...,"In generative modeling, numerous successful ap...",Reviewer_ssMC,1688676041356,1702411081106,6,4,3,3,3,This paper proposes an asymmetric training sch...,574,0,3,0.7740,0.212049
15171,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_UdXn,1688449656890,1702411268900,7,5,3,3,3,"In this paper, authors present regression adju...",1063,1,1,0.8006,0.095406
15172,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_ZgSr,1688485585833,1702411268818,5,3,2,2,2,This paper focuses on estimation of individual...,519,3,2,0.7540,0.099046
15173,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_HGb9,1688665406904,1702411268706,4,3,2,1,3,This paper explores the design and analysis of...,1546,3,9,0.7824,0.090550


In [8]:
# Save the new DataFrame to a different JSON file
out = '/home/ali/Review_Quality_Benchmark/data/processed/openreview_neurips2023_v4.json'
df_reviews.to_json(out, orient='records', indent=4)

# V6

In [9]:
# Read the JSON file as a pandas DataFrame
df_reviews = pd.read_json('/home/ali/Review_Quality_Benchmark/data/processed/openreview_neurips2023_v4.json')

# Display the first few rows of the DataFrame
df_reviews

,submission_id,submission_number,submission_creation_date,submission_authors,submission_title,submission_abstract,reviewer,creation_date,last_modification_date,review_rating,review_confidence,review_soundness,review_presentation,review_contribution,total_review,length_words,citation_count,question_count,mattr,sentiment_polarity
0,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_AvJq,1688368213177,1702411303415,6,3,3,3,3,This paper proposes a model-agnostic framework...,225,0,8,0.7494,0.041667
1,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_7E4k,1688505633161,1702411303319,6,4,3,3,4,"In this paper, the authors present a unique ap...",513,0,1,0.7788,0.107136
2,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_hHZH,1688552936677,1702411303221,6,4,3,3,2,I have read the other reviews and all rebuttal...,650,0,6,0.7975,0.093388
3,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_sVgx,1688657604892,1702411303144,6,4,2,3,3,Paper introduces a framework to detect and cat...,453,0,1,0.8136,0.113038
4,zyZkaqNnpa,15594,1683835167534,"[~Aahlad_Manas_Puli1, ~Lily_H_Zhang1, ~Yoav_Wa...",Don’t blame Dataset Shift! Shortcut Learning d...,Common explanations for shortcut learning assu...,Reviewer_9QKU,1688617232745,1702411520565,6,3,3,3,3,This paper theoretecally and empirically showe...,307,0,0,0.8162,0.024399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15170,00EKYYu3fD,6838,1683719044332,"[~Tianyang_Hu1, ~Fei_Chen8, ~Haonan_Wang1, ~Ji...",Complexity Matters: Rethinking the Latent Spac...,"In generative modeling, numerous successful ap...",Reviewer_ssMC,1688676041356,1702411081106,6,4,3,3,3,This paper proposes an asymmetric training sch...,574,0,3,0.7740,0.212049
15171,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_UdXn,1688449656890,1702411268900,7,5,3,3,3,"In this paper, authors present regression adju...",1063,1,1,0.8006,0.095406
15172,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_ZgSr,1688485585833,1702411268818,5,3,2,2,2,This paper focuses on estimation of individual...,519,3,2,0.7540,0.099046
15173,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_HGb9,1688665406904,1702411268706,4,3,2,1,3,This paper explores the design and analysis of...,1546,3,9,0.7824,0.090550


In [10]:
import csv
import json
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import torch.nn.functional as F

# --- Load SPECTER model ---
model_name = "allenai/specter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


def encoding_text(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=512,
        padding=True
    ).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :]  # [CLS] token
    
    return embeddings


def compute_relevance_score(review_text, title, abstract):
    # Encode document
    doc_emb = encoding_text(f"{title} {abstract}")

    # Encode review text
    review_emb = encoding_text(review_text)
    
    # Compute cosine similarity
    return F.cosine_similarity(doc_emb, review_emb).item()


# Compute similarity score for each row with progress bar
df_reviews['similarity_score'] = [
    compute_relevance_score(row['total_review'], row['submission_title'], row['submission_abstract'])
    for row in tqdm(df_reviews.to_dict('records'), desc="Computing similarity scores")
]

# Display the updated DataFrame
df_reviews

2025-05-08 16:43:24.962472: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746737004.984484 1212053 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746737004.991224 1212053 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746737005.010641 1212053 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746737005.010667 1212053 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746737005.010669 1212053 computation_placer.cc:177] computation placer alr

,submission_id,submission_number,submission_creation_date,submission_authors,submission_title,submission_abstract,reviewer,creation_date,last_modification_date,review_rating,...,review_soundness,review_presentation,review_contribution,total_review,length_words,citation_count,question_count,mattr,sentiment_polarity,similarity_score
0,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_AvJq,1688368213177,1702411303415,6,...,3,3,3,This paper proposes a model-agnostic framework...,225,0,8,0.7494,0.041667,0.941130
1,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_7E4k,1688505633161,1702411303319,6,...,3,3,4,"In this paper, the authors present a unique ap...",513,0,1,0.7788,0.107136,0.947173
2,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_hHZH,1688552936677,1702411303221,6,...,3,3,2,I have read the other reviews and all rebuttal...,650,0,6,0.7975,0.093388,0.891141
3,zyhxRc9bew,10819,1683789038840,"[~Hao_Sun1, ~Boris_van_Breugel2, ~Jonathan_Cra...",What is Flagged in Uncertainty Quantification?...,Uncertainty quantification (UQ) is essential f...,Reviewer_sVgx,1688657604892,1702411303144,6,...,2,3,3,Paper introduces a framework to detect and cat...,453,0,1,0.8136,0.113038,0.854318
4,zyZkaqNnpa,15594,1683835167534,"[~Aahlad_Manas_Puli1, ~Lily_H_Zhang1, ~Yoav_Wa...",Don’t blame Dataset Shift! Shortcut Learning d...,Common explanations for shortcut learning assu...,Reviewer_9QKU,1688617232745,1702411520565,6,...,3,3,3,This paper theoretecally and empirically showe...,307,0,0,0.8162,0.024399,0.946658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15170,00EKYYu3fD,6838,1683719044332,"[~Tianyang_Hu1, ~Fei_Chen8, ~Haonan_Wang1, ~Ji...",Complexity Matters: Rethinking the Latent Spac...,"In generative modeling, numerous successful ap...",Reviewer_ssMC,1688676041356,1702411081106,6,...,3,3,3,This paper proposes an asymmetric training sch...,574,0,3,0.7740,0.212049,0.856897
15171,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_UdXn,1688449656890,1702411268900,7,...,3,3,3,"In this paper, authors present regression adju...",1063,1,1,0.8006,0.095406,0.902801
15172,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_ZgSr,1688485585833,1702411268818,5,...,2,2,2,This paper focuses on estimation of individual...,519,3,2,0.7540,0.099046,0.894719
15173,009LK0vLcY,10107,1683778565991,"[~Mehrdad_Ghadiri2, ~David_Arbour1, ~Tung_Mai1...",Finite Population Regression Adjustment and No...,The design and analysis of randomized experime...,Reviewer_HGb9,1688665406904,1702411268706,4,...,2,1,3,This paper explores the design and analysis of...,1546,3,9,0.7824,0.090550,0.948687


In [11]:
# Save the new DataFrame to a different JSON file
out = '/home/ali/Review_Quality_Benchmark/data/processed/openreview_neurips2023_v5.json'
df_reviews.to_json(out, orient='records', indent=4)

In [ ]:
# import csv
# from tqdm import tqdm
# from convokit import Corpus, download, TextParser, PolitenessStrategies, Classifier, Utterance, Speaker

# # Step 1: Load training corpus
# print("📥 Downloading training corpus...")
# train_corpus = Corpus(filename=download('wiki-politeness-annotated'))

# # Step 2: Load review data and convert to Utterances with dummy speakers
# review_utterances = []

# with open(input_file, mode='r', encoding='utf-8', errors='ignore') as f:
#     reader = list(csv.DictReader(f))
#     for idx, row in tqdm(enumerate(reader), desc="🔧 Preparing Utterances", total=1805):  # Adjust total if needed
#         review_text = row.get("review_text", "").strip()
#         if review_text:
#             dummy_speaker = Speaker(id=f"reviewer_{idx}")
#             review_utterances.append(
#                 Utterance(id=str(idx), text=review_text, speaker=dummy_speaker, meta={"orig_row": row})
#             )

# # Step 3: Build test corpus
# print("📦 Building test corpus...")
# test_corpus = Corpus(utterances=review_utterances)

# # Step 4: Parse
# print("🧠 Parsing utterances...")
# parser = TextParser()
# parser.transform(train_corpus)
# parser.transform(test_corpus)

# # Step 5: Extract politeness strategies
# print("✨ Extracting politeness strategies...")
# ps = PolitenessStrategies()
# ps.transform(train_corpus)
# ps.transform(test_corpus)

# # Step 6: Train classifier
# print("🎓 Training classifier...")
# clf = Classifier(obj_type='utterance', pred_feats=['politeness_strategies'],
#                  labeller=lambda utt: utt.meta.get("Binary") == 1)
# clf.fit(train_corpus)
# clf.transform(test_corpus)

# # Step 7: Summarize results
# print("📈 Summarizing scores...")
# results = clf.summarize(test_corpus)

# # Step 8: Merge back to CSV rows
# print("🧾 Merging scores into CSV...")
# output_rows = []
# fieldnames = list(reader[0].keys())
# if "politeness_score" not in fieldnames:
#     fieldnames.append("politeness_score")

# for utt in tqdm(test_corpus.iter_utterances(), desc="🔗 Assigning Scores"):
#     row = utt.meta["orig_row"]
#     try:
#         score = results.loc[utt.id, "pred_score"]
#         row["politeness_score"] = round(score, 4)
#     except KeyError:
#         row["politeness_score"] = ""
#     output_rows.append(row)

# # Step 9: Save
# print("💾 Saving to review_analysis.csv...")
# with open(input_file, mode='w', newline='', encoding='utf-8', errors='ignore') as f:
#     writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
#     writer.writeheader()
#     writer.writerows(output_rows)

# print("✅ All done! Politeness scores are now in your CSV.")


# SOROUSH's CODE

In [11]:
# TODO: replace with your own path
output_file = "../data/XXXXX.csv"
input_file = output_file

In [12]:
import json
import csv
from datetime import datetime

rows = []

for paper in data:
    paper_id = paper.get("id", "").strip()
    if paper_id.upper() == "UNK" or not paper_id:
        continue

    paper_date_str = paper.get("date", "")
    try:
        paper_date = datetime.strptime(paper_date_str, "%m/%d/%Y")
    except Exception:
        paper_date = None

    for review in paper.get("reviews", []):
        reviewer = review.get("reviewer", "Anonymous").strip()
        review_date_str = review.get("date", "").strip()

        # Clean review text
        review_text = review.get("comment", "")
        review_text = review_text.replace("\n", " ").replace("\r", " ").strip()
        review_suggestion = review.get("suggestion", "")

        length_words = len(review_text.split())

        try:
            review_date = datetime.strptime(review_date_str, "%d/%b/%Y")
            days_to_submit = (review_date - paper_date).days if paper_date else None
        except Exception:
            days_to_submit = None

        rows.append({
            "paper_id": paper_id,
            "reviewer": reviewer,
            "review_date": review_date_str,
            "review_suggestion": review_suggestion,
            "length_words": length_words,
            "days_to_submit": days_to_submit,
            "review_text": review_text
        })

# Save to CSV with proper quoting
with open(output_file, mode='w', newline='', encoding='utf-8', errors='ignore') as f:
    writer = csv.DictWriter(f, fieldnames=rows[0].keys(), quoting=csv.QUOTE_ALL)
    writer.writeheader()
    writer.writerows(rows)

print(f"✅ Cleaned and saved {len(rows)} reviews with full text to review_analysis.csv")



AttributeError: 'Note' object has no attribute 'get'

In [ ]:
%pip install pylats taaled spacy

In [3]:
%pip install pylats taaled spacy
# English models
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_trf

# Spanish models (used as fallback)
!python -m spacy download es_core_news_sm
!python -m spacy download es_dep_news_trf

     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     - -------------------------------------- 0.0/1.5 MB 653.6 kB/s eta 0:00:03
     -------------- ------------------------- 0.5/1.5 MB 5.6 MB/s eta 0:00:01
     ------------------------------------ --- 1.4/1.5 MB 9.5 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/4.6 MB ? eta -:--:--
     ----- ---------------------------------- 0.6/4.6 MB 18.2 MB/s eta 0:00:01
     ------------ --------------------------- 1.4/4.6 MB 17.5 MB/s eta 0:00:01
     -------------------- ------------------- 2.4/4.6 MB 18.8 MB/s eta 0:00:01
     ------------------------------- -------- 3.6/4.6 MB 21.0 MB/s eta 0:00:01
     ---------------------------------------  4.5/4.6 MB 20.7 MB/s eta 0:00:01
     ---------------------------------------- 4

In [ ]:
import csv
from taaled import ld
from pylats import lats
from tqdm import tqdm

output_rows = []

# Read rows
with open(input_file, mode='r', encoding='utf-8', errors='ignore') as f:
    reader = list(csv.DictReader(f))
    fieldnames = list(reader[0].keys())

    # Ensure 'mattr' column exists
    if "mattr" not in fieldnames:
        fieldnames.append("mattr")
    # Drop 'mattr_reason' if it exists
    if "mattr_reason" in fieldnames:
        fieldnames.remove("mattr_reason")

    for row in tqdm(reader, desc="Computing MATTR"):
        review_text = row.get("review_text", "").strip()
        mattr_value = ""

        try:
            cleaned = lats.Normalize(review_text, lats.ld_params_en)
            tokens = cleaned.toks
            mattr_value = f"{ld.lexdiv(tokens).mattr:.4f}"
        except Exception as e:
            mattr_value = ""

        row["mattr"] = mattr_value
        # Remove 'mattr_reason' if it exists in the row
        row.pop("mattr_reason", None)
        output_rows.append(row)

# Write updated file
with open(input_file, mode='w', newline='', encoding='utf-8', errors='ignore') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
    writer.writeheader()
    writer.writerows(output_rows)

print("✅ Clean MATTR values saved to review_analysis.csv")


plotnine has not been installed.
To enable advanced data visualization features, please install plotnine.
Attempting to load spacy model: en_core_web_sm
Successfully loaded spacy model: en_core_web_sm
Attempting to load spacy model: en_core_web_trf
Successfully loaded spacy model: en_core_web_trf
Attempting to load spacy model: es_core_news_sm
Successfully loaded spacy model: es_core_news_sm
Attempting to load spacy model: es_dep_news_trf
Successfully loaded spacy model: es_dep_news_trf


Computing MATTR: 100%|██████████| 1805/1805 [06:18<00:00,  4.76it/s]

✅ Clean MATTR values saved to review_analysis.csv


In [5]:
!pip install transformers torch nltk

###########################
# Apple silicon support
# Uninstall current PyTorch version (if any)
# !pip uninstall torch -y

# Install PyTorch with MPS (Metal Performance Shaders) support
# !pip install torch==2.1.2 torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
###########################

import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Soroush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Soroush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
import csv
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm
from nltk.tokenize import sent_tokenize
import nltk

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("shahrukhx01/bert-mini-finetune-question-detection")
model = AutoModelForSequenceClassification.from_pretrained("shahrukhx01/bert-mini-finetune-question-detection")
model.eval()

output_rows = []

# Load review rows
with open(input_file, mode='r', encoding='utf-8', errors='ignore') as f:
    reader = list(csv.DictReader(f))
    fieldnames = list(reader[0].keys())
    if "question_count" not in fieldnames:
        fieldnames.append("question_count")

    for row in tqdm(reader, desc="Detecting Questions"):
        review_text = row.get("review_text", "")
        question_count = 0

        try:
            sentences = sent_tokenize(review_text)
            for sent in sentences:
                inputs = tokenizer(
                    sent,
                    return_tensors="pt",
                    truncation=True,
                    max_length=64,
                    padding=True
                )
                with torch.no_grad():
                    outputs = model(**inputs)
                    predicted = torch.argmax(outputs.logits, dim=1).item()

                    # Label 0 = question
                    if predicted == 0:
                        question_count += 1
        except Exception as e:
            question_count = ""

        row["question_count"] = question_count
        output_rows.append(row)

# Save updated CSV
with open(input_file, mode='w', newline='', encoding='utf-8', errors='ignore') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
    writer.writeheader()
    writer.writerows(output_rows)

print("✅ Questions counted and saved in review_analysis.csv")


Detecting Questions: 100%|██████████| 1805/1805 [03:15<00:00,  9.22it/s]

✅ Questions counted and saved in review_analysis.csv


In [ ]:
import csv
import re
from tqdm import tqdm

# --- Citation counting logic ---
def count_citations(text):
    citation_patterns = [
        r'\[\d+(?:,\s*\d+)*\]',                         # [1], [1, 2, 3]
        r'\([A-Za-z]+ et al\.,\s*\d{4}\)',               # (Smith et al., 2020)
        r'\(\d{4}[a-z]?\)',                              # (2020), (2020a)
        r'\[[A-Za-z]+\d{4}[a-z]?\]',                     # [Smith2020], [Johnson2021a]
        r'\b(?:doi:|arxiv:|https?://[^\s]+)',             # DOI, arXiv, URLs
    ]
    pattern = '|'.join(citation_patterns)
    matches = re.findall(pattern, text)
    return len(matches)

# --- Load CSV and apply ---
output_rows = []

with open(input_file, mode='r', encoding='utf-8', errors='ignore') as f:
    reader = list(csv.DictReader(f))
    fieldnames = list(reader[0].keys())

    # Update for citation_count
    if "citation_count" not in fieldnames:
        fieldnames.append("citation_count")
    if "has_citation" in fieldnames:
        fieldnames.remove("has_citation")  # Remove old 'has_citation' if needed

    for row in tqdm(reader, desc="Counting Citations"):
        review_text = row.get("review_text", "")
        citation_count = count_citations(review_text)
        row["citation_count"] = citation_count
        output_rows.append(row)

# --- Save updated CSV ---
with open(input_file, mode='w', newline='', encoding='utf-8', errors='ignore') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
    writer.writeheader()
    writer.writerows(output_rows)

print("✅ Citation counts added to review_analysis.csv")


Counting Citations: 100%|██████████| 1805/1805 [00:00<00:00, 12740.13it/s]

✅ Citation counts added to review_analysis.csv


In [4]:
import csv

with open(output_file, mode='r', encoding='utf-8', errors='ignore') as f:
    reader = csv.DictReader(f)
    total = 0
    with_citations = 0

    for row in reader:
        total += 1
        if row.get("citation_count") == "2":
            with_citations += 1

print(f"📄 Total reviews: {total}")
print(f"🔍 Reviews with citations: {with_citations}")
print(f"📊 Percentage: {(with_citations / total * 100):.2f}%")

📄 Total reviews: 1805
🔍 Reviews with citations: 110
📊 Percentage: 6.09%


In [9]:
!pip install textblob
!python -m textblob.download_corpora

Finished.


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Soroush\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Soroush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Soroush\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Soroush\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\Soroush\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\Soroush\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_re

In [ ]:
import csv
from textblob import TextBlob
from tqdm import tqdm

output_rows = []

# Read and process the file
with open(input_file, mode='r', encoding='utf-8', errors='ignore') as f:
    reader = list(csv.DictReader(f))
    fieldnames = list(reader[0].keys())

    # Add new column if not already there
    if "sentiment_polarity" not in fieldnames:
        fieldnames.append("sentiment_polarity")

    for row in tqdm(reader, desc="Analyzing Sentiment"):
        review_text = row.get("review_text", "").strip()
        try:
            blob = TextBlob(review_text)
            sentiment = blob.sentiment.polarity
        except Exception:
            sentiment = ""

        row["sentiment_polarity"] = sentiment
        output_rows.append(row)

# Write updated CSV
with open(input_file, mode='w', newline='', encoding='utf-8', errors='ignore') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
    writer.writeheader()
    writer.writerows(output_rows)

print("✅ Sentiment polarity added to review_analysis.csv")

Analyzing Sentiment: 100%|██████████| 1805/1805 [00:03<00:00, 509.38it/s]


✅ Sentiment polarity added to review_analysis.csv


In [11]:
!pip install convokit
!python -m spacy download en_core_web_sm

  Using cached numpy-2.2.4-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached numpy-2.1.3-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.1.3-cp312-cp312-win_amd64.whl (12.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.59.1 requires numpy<1.27,>=1.22, but you have numpy 2.1.3 which is incompatible.
pywavelets 1.5.0 requires numpy<2.0,>=1.22.4, but you have numpy 2.1.3 which is incompatible.
streamlit 1.32.0 requires numpy<2,>=1.19.3, but you have numpy 2.1.3 which is incompatible.


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB 1.3 MB/s eta 0:00:10
      --------------------------------------- 0.2/12.8 MB 2.4 MB/s eta 0:00:06
     - -------------------------------------- 0.5/12.8 MB 4.0 MB/s eta 0:00:04
     -- ------------------------------------- 0.9/12.8 MB 5.7 MB/s eta 0:00:03
     --- ------------------------------------ 1.2/12.8 MB 6.0 MB/s eta 0:00:02
     ---- ----------------------------------- 1.5/12.8 MB 6.4 MB/s eta 0:00:02
     ------ --------------------------------- 2.1/12.8 MB 7.3 MB/s eta 0:00:02
     ------- -------------------------------- 2.5/12.8 MB 7.2 MB/s eta 0:00:02
     ---------- ----------------------------- 3.5/12.8 MB 8.8 MB/s eta 0:00:02
     ------------- -------------------------- 4.5/12.8 MB 10.2 MB/s eta 0:00:01
     ---------------- ----------------------- 5.1/12.8 MB 10.6 MB/s eta 0:00:01
     ------------------ --------------------- 6.0/12.8 MB

In [ ]:
import csv
from tqdm import tqdm
from convokit import Corpus, download, TextParser, PolitenessStrategies, Classifier, Utterance, Speaker

# Step 1: Load training corpus
print("📥 Downloading training corpus...")
train_corpus = Corpus(filename=download('wiki-politeness-annotated'))

# Step 2: Load review data and convert to Utterances with dummy speakers
review_utterances = []

with open(input_file, mode='r', encoding='utf-8', errors='ignore') as f:
    reader = list(csv.DictReader(f))
    for idx, row in tqdm(enumerate(reader), desc="🔧 Preparing Utterances", total=1805):  # Adjust total if needed
        review_text = row.get("review_text", "").strip()
        if review_text:
            dummy_speaker = Speaker(id=f"reviewer_{idx}")
            review_utterances.append(
                Utterance(id=str(idx), text=review_text, speaker=dummy_speaker, meta={"orig_row": row})
            )

# Step 3: Build test corpus
print("📦 Building test corpus...")
test_corpus = Corpus(utterances=review_utterances)

# Step 4: Parse
print("🧠 Parsing utterances...")
parser = TextParser()
parser.transform(train_corpus)
parser.transform(test_corpus)

# Step 5: Extract politeness strategies
print("✨ Extracting politeness strategies...")
ps = PolitenessStrategies()
ps.transform(train_corpus)
ps.transform(test_corpus)

# Step 6: Train classifier
print("🎓 Training classifier...")
clf = Classifier(obj_type='utterance', pred_feats=['politeness_strategies'],
                 labeller=lambda utt: utt.meta.get("Binary") == 1)
clf.fit(train_corpus)
clf.transform(test_corpus)

# Step 7: Summarize results
print("📈 Summarizing scores...")
results = clf.summarize(test_corpus)

# Step 8: Merge back to CSV rows
print("🧾 Merging scores into CSV...")
output_rows = []
fieldnames = list(reader[0].keys())
if "politeness_score" not in fieldnames:
    fieldnames.append("politeness_score")

for utt in tqdm(test_corpus.iter_utterances(), desc="🔗 Assigning Scores"):
    row = utt.meta["orig_row"]
    try:
        score = results.loc[utt.id, "pred_score"]
        row["politeness_score"] = round(score, 4)
    except KeyError:
        row["politeness_score"] = ""
    output_rows.append(row)

# Step 9: Save
print("💾 Saving to review_analysis.csv...")
with open(input_file, mode='w', newline='', encoding='utf-8', errors='ignore') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
    writer.writeheader()
    writer.writerows(output_rows)

print("✅ All done! Politeness scores are now in your CSV.")


📥 Downloading training corpus...
Dataset already exists at C:\Users\Soroush\.convokit\saved-corpora\wiki-politeness-annotated


🔧 Preparing Utterances: 100%|██████████| 1805/1805 [00:00<00:00, 257788.03it/s]


📦 Building test corpus...
🧠 Parsing utterances...
✨ Extracting politeness strategies...
🎓 Training classifier...
Initialized default classification model (standard scaled logistic regression).
📈 Summarizing scores...
🧾 Merging scores into CSV...


🔗 Assigning Scores: 1805it [00:00, 54323.35it/s]

💾 Saving to review_analysis.csv...
✅ All done! Politeness scores are now in your CSV.


In [ ]:
import csv
import json
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import torch.nn.functional as F

# --- Load SPECTER model ---
model_name = "allenai/specter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

output_rows = []

with open(input_file, mode='r', encoding='utf-8', errors='ignore') as f:
    reader = list(csv.DictReader(f))
    fieldnames = list(reader[0].keys())

    if "similarity_score" not in fieldnames:
        fieldnames.append("similarity_score")

    for row in tqdm(reader, desc="Computing Relevance Score"):
        review_text = row.get("review_text", "")
        paper_id = row.get("paper_id", "").strip()

        try:
            # Find matching entry in data
            matched_entry = next((entry for entry in data if str(entry.get("id", "")).strip() == paper_id), None)

            if matched_entry:
                title = matched_entry.get("title", "")
                abstract = matched_entry.get("abstract", "")
                doc_text = f"{title} {abstract}"

                # Encode document
                doc_inputs = tokenizer(doc_text, return_tensors="pt", truncation=True, padding=True, max_length=512)
                doc_inputs = {k: v.to(device) for k, v in doc_inputs.items()}
                with torch.no_grad():
                    doc_emb = model(**doc_inputs).last_hidden_state[:, 0, :]  # [CLS]

                # Encode review text
                review_inputs = tokenizer(review_text, return_tensors="pt", truncation=True, padding=True, max_length=512)
                review_inputs = {k: v.to(device) for k, v in review_inputs.items()}
                with torch.no_grad():
                    review_emb = model(**review_inputs).last_hidden_state[:, 0, :]  # [CLS]

                # Cosine similarity
                similarity_score = F.cosine_similarity(doc_emb, review_emb).item()
                row["similarity_score"] = similarity_score

            else:
                row["similarity_score"] = ""

        except Exception as e:
            row["similarity_score"] = ""

        output_rows.append(row)

# --- Save updated CSV ---
with open(input_file, mode='w', newline='', encoding='utf-8', errors='ignore') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
    writer.writeheader()
    writer.writerows(output_rows)

print("✅ Relevance scores added to review_analysis.csv")


tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

c:\Users\Soroush\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Soroush\.cache\huggingface\hub\models--allenai--specter. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Computing Relevance Score:   0%|          | 0/1805 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Computing Relevance Score: 100%|██████████| 1805/1805 [00:53<00:00, 33.49it/s]

✅ Relevance scores added to review_analysis.csv


In [ ]:
import csv
import json
from tqdm import tqdm


output_rows = []

with open(input_file, mode='r', encoding='utf-8', errors='ignore') as f:
    reader = list(csv.DictReader(f))
    original_fieldnames = list(reader[0].keys())

    # Insert title and abstract at positions 5 and 6
    new_fieldnames = original_fieldnames[:5] + ["title", "abstract"] + original_fieldnames[5:]

    for row in tqdm(reader, desc="Adding Title and Abstract (Escaping Newlines)"):
        paper_id = row.get("paper_id", "").strip()

        # Find matching entry
        matched_entry = next((entry for entry in data if str(entry.get("id", "")).strip() == paper_id), None)

        if matched_entry:
            title = matched_entry.get("title", "")
            abstract = matched_entry.get("abstract", "")
        else:
            title = ""
            abstract = ""

        # Escape real newlines in title and abstract
        title = title.replace("\r\n", "\\n").replace("\n", "\\n")
        abstract = abstract.replace("\r\n", "\\n").replace("\n", "\\n")

        # Build new row
        new_row = {}
        for idx, field in enumerate(new_fieldnames):
            if field == "title":
                new_row[field] = title
            elif field == "abstract":
                new_row[field] = abstract
            else:
                # Map original fields
                original_field_idx = idx if idx < 5 else idx - 2  # Adjust because we inserted 2 fields
                if original_field_idx < len(original_fieldnames):
                    original_field = original_fieldnames[original_field_idx]
                    new_row[field] = row.get(original_field, "")

        output_rows.append(new_row)

# --- Save updated CSV ---
with open(input_file, mode='w', newline='', encoding='utf-8', errors='ignore') as f:
    writer = csv.DictWriter(f, fieldnames=new_fieldnames, quoting=csv.QUOTE_ALL)
    writer.writeheader()
    writer.writerows(output_rows)

print("✅ Title and Abstract (with clean \\n) added to review_analysis.csv")


Adding Title and Abstract (Escaping Newlines): 100%|██████████| 1805/1805 [00:00<00:00, 20347.07it/s]

✅ Title and Abstract (with clean \n) added to review_analysis.csv


In [ ]:
import csv
from tqdm import tqdm
from datetime import datetime

# Helper: parse dates consistently
def parse_date(date_str):
    try:
        return datetime.strptime(date_str, "%d/%b/%Y")
    except Exception:
        return None

output_rows = []

with open(input_file, mode='r', encoding='utf-8', errors='ignore') as f:
    reader = list(csv.DictReader(f))
    rows = list(reader)
    fieldnames = list(rows[0].keys())

    if "num_days_before_deadline" not in fieldnames:
        fieldnames.append("num_days_before_deadline")

    # First: find latest review_date per paper_id
    latest_review_dates = {}

    for row in rows:
        paper_id = row["paper_id"]
        review_date = parse_date(row["review_date"])

        if paper_id and review_date:
            if paper_id not in latest_review_dates:
                latest_review_dates[paper_id] = review_date
            else:
                if review_date > latest_review_dates[paper_id]:
                    latest_review_dates[paper_id] = review_date

    # Second: compute days before deadline for each review
    for row in tqdm(rows, desc="Computing num_days_before_deadline"):
        paper_id = row["paper_id"]
        review_date = parse_date(row["review_date"])
        deadline_date = latest_review_dates.get(paper_id)

        if review_date and deadline_date:
            days_before_deadline = (deadline_date - review_date).days
            row["num_days_before_deadline"] = days_before_deadline
        else:
            row["num_days_before_deadline"] = ""

        output_rows.append(row)

# --- Save updated CSV ---
with open(input_file, mode='w', newline='', encoding='utf-8', errors='ignore') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
    writer.writeheader()
    writer.writerows(output_rows)

print("✅ num_days_before_deadline added to review_analysis.csv")


Computing num_days_before_deadline: 100%|██████████| 1805/1805 [00:00<00:00, 161750.21it/s]

✅ num_days_before_deadline added to review_analysis.csv
